In [ ]:
# In your setup cell (Cell 1), after mounting drive:
!pip install timm torchvision tifffile imagecodecs


In [ ]:
# Cell 1: Mount Drive & install
from google.colab import drive
drive.mount('/content/drive')

!pip install timm torchvision tifffile


In [ ]:
# Cell 2 ▶ Consolidated imports
import os
import math
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import timm
from torchvision import transforms, models

from tifffile import imread

In [ ]:
# ─── Cell 3: DATASET WITH AGGRESSIVE AUGMENTATION ───────────────────────────────
class LSTDataset(Dataset):
    def __init__(self, df, patches_dir, weather_cols):
        self.df           = df.reset_index(drop=True)
        self.patches_dir  = patches_dir
        self.weather_cols = weather_cols
        self.transform    = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(20),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row  = self.df.loc[idx]
        arr  = imread(os.path.join(self.patches_dir, row["filename"])).astype(np.float32)

        # Image input
        img_np = arr[[1,2,3]].transpose(1,2,0).astype(np.uint8)
        img    = self.transform(img_np)

        # Target LST
        lst    = torch.tensor(arr[0], dtype=torch.float32).unsqueeze(0)
        lst    = F.interpolate(lst.unsqueeze(0), size=(224,224),
                               mode='bilinear', align_corners=False
                              ).squeeze(0)

        # Weather vector
        weather = torch.tensor(
            row[self.weather_cols].values.astype(np.float32)
        )
        return img, weather, lst


In [ ]:
# ─── Cell 4: ViT-UNet WITH SPATIAL DROPOUT ─────────────────────────────────────
class ViTUNet(nn.Module):
    def __init__(self, in_channels=3, weather_dim=5, base_channels=64,
                 num_heads=8, trans_layers=2, trans_dim=512):
        super().__init__()
        def conv_block(in_ch, out_ch, p_drop=0.2):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.Dropout2d(p_drop),
                nn.Conv2d(out_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
            )
        # Encoder with dropout
        self.conv1 = conv_block(in_channels,    base_channels,    p_drop=0.2)
        self.conv2 = conv_block(base_channels,  base_channels*2,  p_drop=0.2)
        self.conv3 = conv_block(base_channels*2,base_channels*4,  p_drop=0.2)
        self.conv4 = conv_block(base_channels*4,base_channels*8,  p_drop=0.2)
        self.conv5 = conv_block(base_channels*8,base_channels*8,  p_drop=0.2)
        self.pool  = nn.MaxPool2d(2)

        # Weather projection & transformer (unchanged)…
        self.weather_proj   = nn.Linear(weather_dim, trans_dim)
        self.bottleneck_proj = nn.Conv2d(base_channels*8, trans_dim, 1)
        self.pos_embed       = nn.Parameter(torch.zeros(1, 14*14, trans_dim))
        self.weather_token   = nn.Parameter(torch.zeros(1,1,trans_dim))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=trans_dim, nhead=num_heads,
            dim_feedforward=trans_dim*4, dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=trans_layers)

        # Decoder upsample & convs (unchanged)…
        self.up4 = nn.ConvTranspose2d(trans_dim,     base_channels*8, 2,2)
        self.up3 = nn.ConvTranspose2d(base_channels*8, base_channels*4, 2,2)
        self.up2 = nn.ConvTranspose2d(base_channels*4, base_channels*2, 2,2)
        self.up1 = nn.ConvTranspose2d(base_channels*2, base_channels,   2,2)
        self.dec4 = conv_block(base_channels*8 + base_channels*8, base_channels*8, p_drop=0.2)
        self.dec3 = conv_block(base_channels*4 + base_channels*4, base_channels*4, p_drop=0.2)
        self.dec2 = conv_block(base_channels*2 + base_channels*2, base_channels*2, p_drop=0.2)
        self.dec1 = conv_block(base_channels   + base_channels,   base_channels,   p_drop=0.2)
        self.final_conv = nn.Conv2d(base_channels, 1, 1)

    def forward(self, x, weather):
        # same forward as before…
        B,C,H,W = x.shape
        e1 = self.conv1(x)
        e2 = self.conv2(self.pool(e1))
        e3 = self.conv3(self.pool(e2))
        e4 = self.conv4(self.pool(e3))
        e5 = self.conv5(self.pool(e4))

        bt      = self.bottleneck_proj(e5)
        N       = bt.shape[2]*bt.shape[3]
        bt_flat = bt.view(B, bt.shape[1], N).permute(0,2,1)
        bt_pos  = bt_flat + self.pos_embed[:, :N, :]
        w_tok   = self.weather_proj(weather).unsqueeze(1) + self.weather_token
        trans   = torch.cat([bt_pos, w_tok], dim=1)
        out_t   = self.transformer(trans)
        spat    = out_t[:, :-1, :]
        fs      = int(math.sqrt(spat.size(1)))
        f_ts    = spat.permute(0,2,1).view(B, bt.shape[1], fs, fs)

        d4 = self.dec4(torch.cat([self.up4(f_ts), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4),  e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3),  e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2),  e1], dim=1))
        return self.final_conv(d1)


In [ ]:
# define which meteorological columns to pull
weather_cols = [
    "air_temp_C",
    "dew_point_C",
    "relative_humidity_percent",
    "wind_speed_m_s",
    "precipitation_in",
]

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv")
for col in weather_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=weather_cols + ["filename"]).reset_index(drop=True)

patches_dir = "/content/drive/MyDrive/PatchedOutput_Cleaned"
dataset     = LSTDataset(df, patches_dir, weather_cols)
train_sz    = int(0.8 * len(dataset))
val_sz      = len(dataset) - train_sz
train_ds, val_ds = random_split(dataset, [train_sz, val_sz])

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True,  num_workers=0, pin_memory=False)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=0, pin_memory=False)

In [ ]:
# ─── Cell 5: INITIALIZE MODEL, SmoothL1Loss, OPTIMIZER & LR SCHEDULER ─────────
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = ViTUNet(in_channels=3, weather_dim=len(weather_cols)).to(device)

# Use SmoothL1 (Huber) loss instead of MSE
loss_fn = nn.SmoothL1Loss()

# Optimizer: only on parameters that require grad
opt = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4,
    weight_decay=1e-2
)

# Reduce LR on plateau of validation loss
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt,
    mode='min',
    factor=0.5,
    patience=3,
    verbose=True
)

scaler = GradScaler()

print("✅ Model, SmoothL1Loss, optimizer & scheduler ready")


In [ ]:
# ─── Cell 6: TRAIN & VALIDATE WITH EARLY STOPPING ──────────────────────────────
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    module="torch\\.cuda\\.amp"
)

patience           = 5
best_val           = float('inf')
epochs_no_improve  = 0
num_epochs         = 10

for epoch in range(1, num_epochs+1):
    # —— Training ——
    model.train()
    train_loss = 0.0
    train_bar  = tqdm(train_loader, desc=f"Epoch {epoch:02d} ▶ Train", leave=False)
    for imgs, weather, masks in train_bar:
        imgs, weather, masks = imgs.to(device), weather.to(device), masks.to(device)
        opt.zero_grad()
        with torch.amp.autocast(device_type="cuda"):
            preds = model(imgs, weather)
            loss  = loss_fn(preds, masks)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        train_loss += loss.item() * imgs.size(0)
        train_bar.set_postfix(smoothl1=f"{loss.item():.4f}")

    avg_train = train_loss / len(train_loader.dataset)
    print(f"Epoch {epoch:02d} ▶ Train SmoothL1: {avg_train:.4f}")

    # —— Validation ——
    model.eval()
    val_loss = 0.0
    val_bar  = tqdm(val_loader, desc=f"Epoch {epoch:02d} ▶ Val  ", leave=False)
    with torch.no_grad():
        for imgs, weather, masks in val_bar:
            imgs, weather, masks = imgs.to(device), weather.to(device), masks.to(device)
            with torch.amp.autocast(device_type="cuda"):
                preds = model(imgs, weather)
                loss  = loss_fn(preds, masks)

            val_loss += loss.item() * imgs.size(0)
            val_bar.set_postfix(smoothl1=f"{loss.item():.4f}")

    avg_val = val_loss / len(val_loader.dataset)
    print(f"           Val   SmoothL1: {avg_val:.4f}\n")

    # Scheduler step and early stopping
    scheduler.step(avg_val)
    if avg_val < best_val:
        best_val          = avg_val
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"↳ Early stopping at epoch {epoch}")
            break
